# Builds a LangChain chatbot

In [2]:
import enum
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr
import logging as log
import os
import pprint

In [3]:
log.basicConfig(level=log.INFO, format='%(asctime)s [%(levelname)5s] %(message)s',datefmt='%H:%M:%S')

In [4]:
import os

class LlmVendor( enum.Enum):
    Google=1,
    OpenAI=2

class LlmModel( enum.Enum):
    GeminiFlash=1,
    GeminiPro=2
    OpenAI=3
    
class LlmModelFactory():
    def __init__(self, model: LlmModel, vendor: LlmVendor, envVar:str):
        self._model = model
        self._vendor= vendor
        self._envVar= envVar

        if not envVar in os.environ:
            raise ValueError(f'{envVar} not found in the environment.')

    @staticmethod
    def createGoogleGeminiFlash(): 
        return LlmModelFactory(model= LlmModel.GeminiFlash,  
                               vendor= LlmVendor.Google,
                               envVar='GOOGLE_API_KEY')
    @staticmethod
    def modelName( llmModel: LlmModel):
        match llmModel:
            case LlmModel.GeminiFlash:
                return "gemini-1.5-flash"
            case LlmModel.GeminiPro:     
                return "gemini-1.5-pro"
        
    def __str__(self):
        return f'(model:{self._model}, vendor:{self._vendor}, envVar:{self._envVar})'

    def createChatModel(self, 
                       maxOutputTokens=250,
                       temperature=0.0):
        match self._vendor:
            case LlmVendor.Google:
                return ChatGoogleGenerativeAI(model= LlmModelFactory.modelName(self._model), 
                                              max_output_tokens=maxOutputTokens, 
                                              temperature=temperature)
                

llmModelFactory= LlmModelFactory.createGoogleGeminiFlash()
print(llmModelFactory)

(model:LlmModel.GeminiFlash, vendor:LlmVendor.Google, envVar:GOOGLE_API_KEY)


In [5]:
model = llmModelFactory.createChatModel()

In [6]:
messages= [('system','You are a helpful assistant. That can translate from English to {language}'),
           ('human','Translate the following text:\n{text}')]

In [7]:
chain = ChatPromptTemplate(messages) | model | StrOutputParser()

In [8]:
result= chain.invoke({"language": "Italian", "text": "hi!"})

In [9]:
result

'Ciao!'

## Other test

In [10]:
chainModelToParser = model | StrOutputParser()

In [11]:
messages= [('system','You are a helpful assistant. That can translate from English to {language}'),
           ('human','Translate the following text:\n{text}')]
promptTemplate= ChatPromptTemplate(messages)
chain= promptTemplate | chainModelToParser

In [12]:
result= chain.invoke({"language": "Italian", "text": "How are you?"})
result

'How are you?  translates to:\n\n**Come stai?** (informal, singular)  or  **Come state?** (formal, singular or plural)'

# Chatbot. It runs until the user types 'quit'

In [13]:
messages= [('system','You are a polite and helpful assistant.')]
chainModelToParser = model | StrOutputParser()

while True:
    for (role,message) in messages:
        print(f'Role:   {role}')
        print(f'Message:{message}\n')
    print('Enter the next prompt. quit to terminate:')
    userPrompt= input()
    if 'quit'==userPrompt.strip().lower():
        break
    messages.append( ('human',userPrompt))  
    promptTemplate= ChatPromptTemplate(messages)
    chain= promptTemplate | chainModelToParser
    result= chain.invoke({})
    print(result)
    messages.append( ('ai',result))  

Role:   system
Message:You are a polite and helpful assistant.

Enter the next prompt. quit to terminate:


 quit


## Same chatbot, but with Gradio

In [30]:
messages= [('system','You are a polite and helpful assistant.')]
chainModelToParser = model | StrOutputParser()
chatHistory=[]

#----------------------------------------------------------------------------------
def chatBot( userPrompt):
    global chatHistory
    global messages
    
    errMessage = str()
    try:
        messages.append( ('human',userPrompt))  
        
        for (role,message) in messages:
            log.info(f'Role:{role} {message}')
        
        promptTemplate= ChatPromptTemplate(messages)
        chain= promptTemplate | chainModelToParser
        result= chain.invoke({})
        messages.append( ('ai',result))  
        
        chatHistory = [ f'{role}: {message}' for (role,message) in messages]
        textChatHistory= '\n'.join(chatHistory)
        return (result, textChatHistory, errMessage)
        
        
    except Exception as exc:
        log.error(f'Exception detected:{exc}')
        return ("", textChatHistory, str(exc))

chatbotUi = gr.Interface(fn=chatBot, 
                        inputs=[gr.Textbox(label='User Prompt', show_label=True, lines=5)],
                        outputs=[gr.Textbox(label='LLM Response', show_label=True),
                                gr.Textbox(label='Chat History', show_label=True,lines=10),
                                gr.Textbox(label='Error', show_label=True)],
                        title='Chatbot',
                        flagging_mode='never')
chatbotUi.launch()        

10:35:06 [ INFO] HTTP Request: GET http://127.0.0.1:7870/gradio_api/startup-events "HTTP/1.1 200 OK"
10:35:06 [ INFO] HTTP Request: HEAD http://127.0.0.1:7870/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


10:35:06 [ INFO] HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


10:04:17 [ INFO] HTTP Request: GET http://127.0.0.1:7862/gradio_api/startup-events "HTTP/1.1 200 OK"
10:04:17 [ INFO] HTTP Request: HEAD http://127.0.0.1:7862/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


10:04:18 [ INFO] HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
10:04:38 [ INFO] Role:system You are a polite and helpful assistant.
10:04:38 [ INFO] Role:human Who is the current Italian President?

10:05:06 [ INFO] Role:system You are a polite and helpful assistant.
10:05:06 [ INFO] Role:human Who is the current Italian President?

10:05:06 [ INFO] Role:human Had he a brother?

10:05:35 [ INFO] Role:system You are a polite and helpful assistant.
10:05:35 [ INFO] Role:human Who is the current Italian President?

10:05:35 [ INFO] Role:human Had he a brother?

10:05:35 [ INFO] Role:human Who is the sister? Her name and further news about her?

10:06:11 [ INFO] Role:system You are a polite and helpful assistant.
10:06:11 [ INFO] Role:human Who is the current Italian President?

10:06:11 [ INFO] Role:human Had he a brother?

10:06:11 [ INFO] Role:human Who is the sister? Her name and further news about her?

10:06:11 [ INFO] Role:human Ok, at least some public info